In [63]:
# Import Libraries
import arcpy
import requests
import os
from zipfile import ZipFile
import numpy as np

In [45]:
# Define local directory
local_directory = r"E:\coursework\ARCGISI\LAB3\data"
# Create the local directory if it doesn't exist
if not os.path.exists(local_directory):
    os.makedirs(local_directory)
    
arcpy.env.workspace = r"E:\coursework\ARCGISI\LAB3\data"

## Part I

### Download Data

In [47]:
# Downloads DEM
dem_url = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'
response = requests.get(dem_url)
file_name = os.path.join(local_directory, dem_url.split("/")[-1])

with open(file_name, "wb") as f:
    f.write(response.content)
    
with ZipFile(file_name, "r") as zip_ref:
    zip_ref.extractall(local_directory)

In [48]:
# Reproject dem file
sr = arcpy.SpatialReference(4326)
in_raster = os.path.join(arcpy.env.workspace,'elev_30m_digital_elevation_model.gdb','digital_elevation_model_30m')
arcpy.management.ProjectRaster(in_raster, 'dem_reproject',sr)

<Result 'E:\\coursework\\ARCGISI\\Lab2\\data\\dem_reproject'>

In [51]:
# Clip dem file
arcpy.management.Clip('dem_reproject','-92.18 44.20 -92.00 44.00','dem_final')

<Result 'E:\\coursework\\ARCGISI\\Lab2\\data\\dem_final'>

In [ ]:
# Slope
slope = arcpy.sa.Slope("dem_final", "DEGREE", 1, "PLANAR", "METER")
slope.save('slope')

In [57]:
# Downloads LC
landcover_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip"
response = requests.get(landcover_url)
file_name = os.path.join(local_directory, landcover_url.split("/")[-1])

with open(file_name, "wb") as f:
    f.write(response.content)
    
with ZipFile(file_name, "r") as zip_ref:
    zip_ref.extractall(local_directory)

In [59]:
sr = arcpy.SpatialReference(4326)
arcpy.management.ProjectRaster('NLCD_2019_Land_Cover.tif', 'landcover_reproject.tif',sr)

<Result 'E:\\coursework\\ARCGISI\\Lab2\\data\\landcover_reproject.tif'>

In [60]:
# Clip dem file
arcpy.management.Clip('landcover_reproject.tif','-92.18 44.20 -92.00 44.00','landcover_final.tif')

<Result 'E:\\coursework\\ARCGISI\\Lab2\\data\\landcover_final.tif'>

### Modeling

In [78]:
# Standarization slope
rMinResult = arcpy.GetRasterProperties_management('slope', "MINIMUM")
rMin = rMinResult.getOutput(0)

rMaxResult = arcpy.GetRasterProperties_management('slope', "MAXIMUM")
rMax = rMaxResult.getOutput(0)

outRaster = (arcpy.Raster('landcover_final') - float(rMin))/(float(rMax) - float(rMin))
outRaster.save('slope_sd')

In [101]:
# Reclassify landcover
remap = arcpy.sa.RemapValue([
    [11, 1], 
    [21, 0],
    [22, 0],
    [23, 0],
    [24, 0],
    [31, 1],
    [41, 1],
    [42, 1],
    [43, 1],
    [52, 1],
    [71, 1],
    [81, 1],
    [82, 1],
    [90, 1],
    [95, 1]
])

outRaster = arcpy.sa.Reclassify("landcover_final.tif", "Value", remap,'NODATA')
outRaster.save('landcover_sd')

In [117]:
# Create Origin-Destination
point1 = arcpy.Point(-92.148796, 44.127985)  # Origin
point2 = arcpy.Point(-92.044817, 44.054470)  # Destination

arcpy.CreateFeatureclass_management(arcpy.env.workspace, "od", "POINT", spatial_reference=sr)
arcpy.AddField_management("od.shp", "Variable", "Text")

<Result 'E:\\coursework\\ARCGISI\\Lab2\\data\\od.shp'>

In [118]:

with arcpy.da.InsertCursor("od.shp", ["SHAPE@", "Variable"]) as cursor:
    cursor.insertRow([point1, 'Origin'])
    cursor.insertRow([point2, 'Destination'])

del cursor

In [134]:
# Create Several Cost Surfaces to Test Different Model Weights
for i in np.array([0.1, 0.5, 0.9]):
    # Set Weights
    slope_weight = i
    landcover_weight = 1-i
    
    # Calculate Cost and Save as New Raster
    cost = arcpy.Raster('slope_sd') * float(slope_weight) + arcpy.Raster('landcover_sd') * float(landcover_weight)
    cost_surface_filename = 'cost_surface_slope' + str(i) + '_lc' + str(1-i) +'.tif'
    cost.save(cost_surface_filename)

    # Create Cost Path
    path_filename = 'final_path_slope' + str(i) + '_lc' + str(1-i) 
    arcpy.sa.OptimalRegionConnections("od.shp", 'POLYLINE',in_cost_raster = cost,out_neighbor_paths = path_filename)